In [1]:
!hpc

connect to node gpu02 of hpc provided by BDSRC


In [2]:
from sqlalchemy import create_engine


engine = create_engine('mysql+pyodbc://telecom')
%load_ext sql
%sql engine
%config SqlMagic.autopandas = True

Found pyproject.toml from '/Users/jacky/github/telecom'

In [3]:
%sql SHOW tables

Running query in 'mysql+pyodbc://telecom'

10 rows affected.

,Tables_in_telecom
0,cinfo_xy
1,sample_sight_201308
2,sample_user_201308
3,serv_acct_item_0838_201308
4,serv_acct_item_0838_201406
5,tb_asz_cdma_0838_201308
6,tb_asz_cdma_0838_201406
7,tower
8,tower_center
9,tower_hot_201308


In [25]:
%%sql

DESC serv_acct_item_0838_201308

Running query in 'mysql+pyodbc://telecom'

20 rows affected.

,Field,Type,Null,Key,Default,Extra
0,SERV_ID,varchar(8),YES,MUL,None,
1,ACC_NBR,varchar(64),YES,,None,
2,TRAFFIC,int,YES,,None,
3,MONTH_NO,varchar(6),YES,,None,
4,CHARGE,int,YES,,None,
5,DURATION,int,YES,,None,
6,ACCT_ITEM_TYPE_CODE,varchar(6),YES,,None,
7,START_TIME,varchar(19),YES,,None,
8,END_TIME,varchar(19),YES,,None,
9,ETL_TYPE_ID,varchar(2),YES,,None,


# Calling Records

## Variables

- SERV_ID: user_id, can also be found in user information with the same column name
- ACC_NBR: cellphone number, can also be found in user infromation named as MSISDN
    - record the cellphone number of a user who is the client of the telecom company
- ACCT_ITEM_TYPE_CODE: 
- ETL_TYPE_ID: 
- CALLING_AREA_CODE, CALLED_AREA_CODE: determine the abstraction location of the calling and called phone call
- CALLING_NBR, CALLED_NBR: anonymous processed calling and called phone number
- LAC: local area code(most of them are outliers)
- CELL_ID: can be used for joing tower infromation on CELL16

- outlier
    - ACCT_ITEM_TYPE_CODE: -1
    - START_TIME, END_TIME: NOT LIKE '____/%/% %:__:__'
    - CALLING_AREA_CODE, CALLED_AREA_CODE: -1
    - CALLING_NBR, CALLED_NBR: NOT LIKE 'NotMobile_%'
    - LAC: -1, 0
    - CELL_ID: -1, 0

## Quickly look through the variables

In [5]:
%%sql

SELECT *
FROM serv_acct_item_0838_201308
LIMIT 5

Running query in 'mysql+pyodbc://telecom'

5 rows affected.

,SERV_ID,ACC_NBR,TRAFFIC,MONTH_NO,CHARGE,DURATION,ACCT_ITEM_TYPE_CODE,START_TIME,END_TIME,ETL_TYPE_ID,CALLED_AREA_CODE,CALLED_NBR,LAC,CELL_ID,RECV_TRAFFIC,CALLING_AREA_CODE,CALLING_NBR,SEND_TRAFFIC,BILLING_DURATION,CALLS
0,a4zlm2z0,a1cnffy9,0,201308,12,0,020709,2013/8/1 0:00:12,2013/8/1 0:00:12,23,-1,NotMobile_-1,-1,-1,0,-1,NotMobile_-1,0,0,1
1,a4zlm2z0,a1cnffy9,0,201308,12,1,020709,2013/8/1 0:03:03,2013/8/1 0:03:04,23,-1,NotMobile_-1,-1,-1,0,-1,NotMobile_-1,0,0,1
2,a4zlm2z0,a1cnffy9,0,201308,12,0,020709,2013/8/1 0:05:52,2013/8/1 0:05:52,23,-1,NotMobile_-1,-1,-1,0,-1,NotMobile_-1,0,0,1
3,a4zlm2z0,a1cnffy9,0,201308,12,0,020709,2013/8/1 0:08:44,2013/8/1 0:08:44,23,-1,NotMobile_-1,-1,-1,0,-1,NotMobile_-1,0,0,1
4,a4zlm2z0,a1cnffy9,0,201308,12,0,020709,2013/8/1 0:11:45,2013/8/1 0:11:45,23,-1,NotMobile_-1,-1,-1,0,-1,NotMobile_-1,0,0,1


## Exception Detection

- DURATION doesn't have exception

In [4]:
%%sql

SELECT
    COUNT(*)
FROM serv_acct_item_0838_201308
WHERE DURATION < 0

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,COUNT(*)
0,0


- exception of ACCT_ITEM_CODE is -1

In [11]:
%%sql

SELECT 
    ACCT_ITEM_TYPE_CODE,
    COUNT(*)
FROM serv_acct_item_0838_201308
GROUP BY  ACCT_ITEM_TYPE_CODE
ORDER BY COUNT(*) DESC

Running query in 'mysql+pyodbc://telecom'

48 rows affected.

,ACCT_ITEM_TYPE_CODE,COUNT(*)
0,170211,55828623
1,170209,43815325
2,160201,25888559
3,160101,24548848
4,160204,3484611
5,160501,3392784
6,170205,2594883
7,170202,2450229
8,170208,2262250
9,160603,2089841


- There are some invalid time foramt

In [14]:
%%sql

SELECT COUNT(*)
FROM serv_acct_item_0838_201308
WHERE 
    START_TIME NOT LIKE '____/%/% %:__:__' 
    OR
    END_TIME NOT LIKE '____/%/% %:__:__'

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,COUNT(*)
0,44849841


- no exception in ETL_TYPE_ID

In [5]:
%%sql

SELECT
    ETL_TYPE_ID AS etl_type_id,
    COUNT(*)
FROM serv_acct_item_0838_201308
GROUP BY etl_type_id

Running query in 'mysql+pyodbc://telecom'

8 rows affected.

,etl_type_id,COUNT(*)
0,21,60823665
1,22,60428727
2,23,4555058
3,24,43993499
4,31,50138
5,32,145182
6,33,6820
7,34,93054


- The CALLING_AREA_CODE and CALLED_AREA_CODES consist of 3 or 4 digit
- 0838 has the highest frequency

In [2]:
%%sql

SELECT
    CALLING_AREA_CODE,
    COUNT(*)
FROM serv_acct_item_0838_201308
WHERE CALLING_AREA_CODE != -1
GROUP BY CALLING_AREA_CODE
ORDER BY COUNG(*) DESC

Running query in 'mysql+pyodbc://telecom'

510 rows affected.

,calling_area_code,count
0,0838,96205674
1,028,3371312
2,0816,682887
3,023,292693
4,010,155611
...,...,...
505,00961,1
506,00973,1
507,08029,1
508,08071,1


In [5]:
%%sql

SELECT
    CALLING_AREA_CODE,
    COUNT(*)
FROM serv_acct_item_0838_201308
WHERE CALLING_AREA_CODE != -1
GROUP BY CALLING_AREA_CODE
ORDER BY COUNG(*) DESC
LIMIT 20

Running query in 'mysql+pyodbc://telecom'

20 rows affected.

,CALLING_AREA_CODE,count
0,0838,96205674
1,028,3371312
2,0816,682887
3,023,292693
4,010,155611
5,021,155577
6,0817,127257
7,0755,110263
8,0825,106029
9,0833,99333


In [9]:
%%sql

SELECT
    CALLING_AREA_CODE,
    COUNT(*)
FROM serv_acct_item_0838_201308
WHERE CALLING_AREA_CODE != -1 AND LENGTH(CALLING_AREA_CODE) < 3
GROUP BY CALLING_AREA_CODE
ORDER BY COUNT(*) DESC

Running query in 'mysql+pyodbc://telecom'

""


In [11]:
%%sql

SELECT
    CALLED_AREA_CODE,
    COUNT(*) AS count
FROM serv_acct_item_0838_201308
WHERE CALLED_AREA_CODE != -1
GROUP BY CALLED_AREA_CODE
ORDER BY count DESC
LIMIT 20

Running query in 'mysql+pyodbc://telecom'

20 rows affected.

,CALLED_AREA_CODE,count
0,0838,93860363
1,028,3794338
2,0816,761832
3,023,320694
4,0817,147813
5,010,134660
6,0825,129196
7,021,119936
8,0833,107266
9,0755,101168


In [14]:
%%sql

SELECT
    CALLED_AREA_CODE,
    COUNT(*) AS count
FROM serv_acct_item_0838_201308
WHERE CALLED_AREA_CODE != -1 AND LENGTH(CALLED_AREA_CODE) < 3
GROUP BY CALLED_AREA_CODE
ORDER BY count DESC

Running query in 'mysql+pyodbc://telecom'

""


- the exception are -1, 0 fro LAC
- most of the observations's LAC is exception

In [6]:
%%sql

SELECT
    LAC AS lac,
    COUNT(*)
FROM serv_acct_item_0838_201308
GROUP BY lac

Running query in 'mysql+pyodbc://telecom'

115 rows affected.

,lac,COUNT(*)
0,-1,109329163
1,0,2336979
2,00000,46453
3,00001,214
4,00003,79
...,...,...
110,0264A,1
111,02A77,1
112,03133,1
113,09000,1


- most of the observation's CELL_ID is exception, namely the value is -1 or 0

In [15]:
%%sql

SELECT
    CELL_ID,
    COUNT(*)
FROM serv_acct_item_0838_201308
GROUP BY CELL_ID

Running query in 'mysql+pyodbc://telecom'

57001 rows affected.

,CELL_ID,COUNT(*)
0,-1,48985582
1,0,2336698
2,0000,47155
3,0001,2184
4,0002,1151
...,...,...
56996,FFF2,32
56997,FFF3,2
56998,FFF7,249
56999,FFF9,2


##  try to figure out the period of the records

In [3]:
%%sql

SELECT DISTINCT
    MONTH_NO
FROM serv_acct_item_0838_201308

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,MONTH_NO
0,201308


In [6]:
%%sql 

SELECT
    date, 
    COUNT(*) AS count
FROM (
    SELECT SUBSTRING_INDEX(START_TIME,' ',1) AS date
    FROM serv_acct_item_0838_201308
) AS subquery
GROUP BY date

Running query in 'mysql+pyodbc://telecom'

31 rows affected.

,date,count
0,2013/8/1,6179713
1,2013/8/10,5070191
2,2013/8/11,5010373
3,2013/8/12,5501876
4,2013/8/13,5760491
5,2013/8/14,5413238
6,2013/8/15,5415015
7,2013/8/16,5758206
8,2013/8/17,5054504
9,2013/8/18,4982765


In [7]:
%%sql 

SELECT
    date, 
    COUNT(*) AS count
FROM (
    SELECT SUBSTRING_INDEX(START_TIME,' ',1) AS date
    FROM serv_acct_item_0838_201406
) AS subquery
GROUP BY date

Running query in 'mysql+pyodbc://telecom'

33 rows affected.

,date,count
0,2014/5/29,3
1,2014/5/30,1
2,2014/5/31,1396
3,2014/6/1,6079751
4,2014/6/10,5354899
5,2014/6/11,5387891
6,2014/6/12,5223904
7,2014/6/13,5434661
8,2014/6/14,5064052
9,2014/6/15,5159278


## Inspect the meaning of some column

In [3]:
%config SqlMagic.named_parameters=True

In [15]:
%%sql

CREATE TABLE clean_cdr(
    SELECT
        B.CI_BRANCH                                                                   AS branch,
        B.CI_DISTRICT                                                                 AS district,
        A.SERV_ID                                                                     AS id,
        A.ACC_NBR                                                                     AS client_nbr, 
        CONVERT(SUBSTRING_INDEX(SUBSTRING_INDEX(A.END_TIME,' ',-1),':',1),UNSIGNED)   AS time,
        DAYOFWEEK(SUBSTRING_INDEX(A.END_TIME,' ',1))                                  AS day_of_week,
        A.CALLING_NBR                                                                 AS calling_nbr,
        A.CALLING_AREA_CODE                                                           AS calling_area_code,
        A.DURATION                                                                    AS duration,
        A.CALLED_AREA_CODE                                                            AS called_area_code,
        A.CALLED_NBR                                                                  AS called_nbr, 
        A.CHARGE                                                                      AS charge, 
        A.ACCT_ITEM_TYPE_CODE                                                         AS acct_item_type_code, 
        A.ETL_TYPE_ID                                                                 AS etl_type_id,
        A.CELL_ID                                                                     AS cell_id
    FROM serv_acct_item_0838_201308 AS A 
    LEFT JOIN tb_asz_cdma_0838_201308 B ON A.SERV_ID=B.SERV_ID AND A.ACC_NBR=B.MSISDN
    WHERE 
        A.ACCT_ITEM_TYPE_CODE != -1                                                    AND
        (A.START_TIME LIKE "____/%/% %:__:__" AND A.END_TIME LIKE "____/%/% %:__:__")  AND
        (A.CALLING_NBR NOT LIKE 'NotMobile_%' AND A.CALLED_NBR NOT LIKE 'NotMobile_%') AND
        (A.CALLING_AREA_COdE != '-1' AND A.CALLED_AREA_CODE != '-1')                   AND
        (A.LAC != '-1' AND A.LAC != '0')                                               AND
        (A.CELL_ID != '-1' AND A.CELL_ID != '0')
)

Running query in 'mysql+pyodbc://telecom'

53121025 rows affected.

""


In [16]:
%sql desc clean_cdr

Running query in 'mysql+pyodbc://telecom'

15 rows affected.

,Field,Type,Null,Key,Default,Extra
0,branch,varchar(255),YES,,None,
1,district,varchar(255),YES,,None,
2,id,varchar(8),YES,,None,
3,client_nbr,varchar(64),YES,,None,
4,time,bigint unsigned,YES,,None,
5,day_of_week,int,YES,,None,
6,calling_nbr,varchar(64),YES,,None,
7,calling_area_code,varchar(8),YES,,None,
8,duration,int,YES,,None,
9,called_area_code,varchar(8),YES,,None,


In [10]:
%sql SELECT * FROM clean_cdr LIMIT 60

Running query in 'mysql+pyodbc://telecom'

60 rows affected.

,branch,district,id,client_nbr,time,day_of_week,calling_nbr,calling_area_code,duration,called_area_code,called_nbr,charge,acct_item_type_code,etl_type_id,cell_id
0,德阳市市辖区,德阳现业,8kvi2lt9,eh2djdgr,9,5,eh2djdgr,0838,100,0838,631wl9ab,20,160201,21,34C1
1,德阳市市辖区,德阳现业,8kvi2lt9,eh2djdgr,9,5,akmslxqx,0838,20,0838,eh2djdgr,0,160101,21,34C1
2,德阳市市辖区,德阳现业,9vei43io,itp9jhv5,9,5,itp9jhv5,0838,135,0838,2fsek3dy,30,160201,21,3473
3,德阳市市辖区,德阳现业,9vei43io,itp9jhv5,9,5,itp9jhv5,0838,22,0838,4gk8anhv,10,160201,21,3473
4,德阳市市辖区,德阳现业,8kvi2lt9,eh2djdgr,9,5,4dbnlczn,0838,11,0838,eh2djdgr,0,160101,21,34C1
5,德阳市市辖区,德阳现业,9vei43io,itp9jhv5,10,5,2fsek3dy,0838,91,0838,itp9jhv5,0,160101,21,3473
6,德阳市市辖区,德阳现业,9vei43io,itp9jhv5,10,5,itp9jhv5,0838,22,0838,ixrojvwp,10,160201,21,3473
7,德阳市市辖区,德阳现业,90m62trl,g6vxjd2w,11,5,7wps2c9o,0838,29,0838,g6vxjd2w,0,160101,21,3740
8,德阳市市辖区,德阳现业,90m62trl,g6vxjd2w,10,5,7wps2c9o,0838,19,0838,g6vxjd2w,0,160101,21,3BD0
9,德阳市市辖区,德阳现业,90m62trl,g6vxjd2w,11,5,d59t2d4b,0838,11,0838,g6vxjd2w,0,160101,21,34C0


- the isn't case that the client number is neither calling number nor called number(occurs in raw data)

In [19]:
%sql SELECT COUNT(*) FROM clean_cdr WHERE client_nbr != calling_nbr AND client_nbr != called_nbr

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,COUNT(*)
0,0


In [7]:
%%sql

SELECT
    branch,
    COUNT(*)
FROM clean_cdr
GROUP BY branch

Running query in 'mysql+pyodbc://telecom'

8 rows affected.

,branch,COUNT(*)
0,None,219390
1,中江县,8007501
2,什邡市,7339788
3,广汉市,9933897
4,德阳市市辖区,18487498
5,德阳市未知营业区,9668
6,绵竹市,6531970
7,罗江县,2591313


- distribution of the phone call's end time

In [13]:
%%sql

SELECT
    time,
    COUNT(*)
FROM clean_cdr
GROUP BY time
ORDER BY time

Running query in 'mysql+pyodbc://telecom'

24 rows affected.

,time,COUNT(*)
0,0,290954
1,1,142586
2,2,84071
3,3,59412
4,4,58319
5,5,108623
6,6,514265
7,7,1527213
8,8,3256009
9,9,4338714


- distribution of phone call's day of week

In [14]:
%%sql

SELECT
    day_of_week,
    COUNT(*)
FROM clean_cdr
GROUP BY day_of_week
ORDER BY day_of_week

Running query in 'mysql+pyodbc://telecom'

7 rows affected.

,day_of_week,COUNT(*)
0,1,6341472
1,2,7043188
2,3,7005202
3,4,6832250
4,5,8735949
5,6,8834986
6,7,8327978


In [25]:
%%sql 
SELECT  
    acct_item_type_code, COUNT(*)
FROM clean_cdr
GROUP BY acct_item_type_code

Running query in 'mysql+pyodbc://telecom'

9 rows affected.

,acct_item_type_code,COUNT(*)
0,030105,6088
1,160101,22617798
2,160201,23017227
3,160203,5
4,160204,3205179
5,160501,2099771
6,160603,1248301
7,160604,926654
8,170212,2


In [24]:
%sql SELECT DISTINCT etl_type_id FROM clean_cdr

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,etl_type_id
0,21


- the reamined area codes no matter the calling or called are all 0838 which indicates that all phone calls are within deyang

In [18]:
%%sql

SELECT
    client_nbr, calling_area_code, COUNT(*)
FROM clean_cdr
WHERE client_nbr = calling_nbr
GROUP BY client_nbr, calling_area_code

Running query in 'mysql+pyodbc://telecom'

417043 rows affected.

,client_nbr,calling_area_code,COUNT(*)
0,27h4lxfq,0838,243
1,27h4lxla,0838,4
2,27h4lxo2,0838,52
3,27h4lxqu,0838,34
4,27h4lxtm,0838,40
...,...,...,...
417038,njzp3nf3,0838,4
417039,njzp3nkn,0838,115
417040,njzp3nnf,0838,14
417041,njzp3nsz,0838,41


In [20]:
%%sql

SELECT DISTINCT
    calling_area_code
FROM(
    SELECT
        client_nbr, calling_area_code, COUNT(*)
    FROM clean_cdr
    WHERE client_nbr = calling_nbr
    GROUP BY client_nbr, calling_area_code
) AS subquery

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,calling_area_code
0,0838


In [21]:
%%sql

SELECT
    client_nbr, called_area_code, COUNT(*)
FROM clean_cdr
WHERE client_nbr = called_nbr
GROUP BY client_nbr, called_area_code

Running query in 'mysql+pyodbc://telecom'

409098 rows affected.

,client_nbr,called_area_code,COUNT(*)
0,27h4lxfq,0838,75
1,27h4lxla,0838,9
2,27h4lxo2,0838,118
3,27h4lxqu,0838,41
4,27h4lxtm,0838,26
...,...,...,...
409093,njzp3nf3,0838,9
409094,njzp3nkn,0838,138
409095,njzp3nnf,0838,3
409096,njzp3nsz,0838,73


In [22]:
%%sql

SELECT DISTINCT
    called_area_code
FROM(
    SELECT
        client_nbr, called_area_code, COUNT(*)
    FROM clean_cdr
    WHERE client_nbr = called_nbr
    GROUP BY client_nbr, called_area_code
) AS subquery

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,called_area_code
0,0838


## Check the meaning of ACC_NBR
- can't find the construction rule for client nubmer because when both the calling number and called number are client, client number can be calling number or called number

In [2]:
%%sql

SELECT COUNT(*)
FROM clean_cdr
WHERE 
    calling_nbr IN (SELECT MSISDN FROM tb_asz_cdma_0838_201308) AND
    called_nbr IN (SELECT MSISDN FROM tb_asz_cdma_0838_201308)

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,COUNT(*)
0,13244425


In [4]:
%%sql

SELECT COUNT(*)
FROM clean_cdr
WHERE 
    calling_nbr IN (SELECT MSISDN FROM tb_asz_cdma_0838_201308) AND
    called_nbr IN (SELECT MSISDN FROM tb_asz_cdma_0838_201308) AND
    client_nbr = calling_nbr

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,COUNT(*)
0,6597536


In [5]:
%%sql

SELECT COUNT(*)
FROM clean_cdr
WHERE 
    calling_nbr IN (SELECT MSISDN FROM tb_asz_cdma_0838_201308) AND
    called_nbr IN (SELECT MSISDN FROM tb_asz_cdma_0838_201308) AND
    client_nbr = called_nbr

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,COUNT(*)
0,6647008


In [16]:
%%sql

SELECT calling_nbr, called_nbr, COUNT(*) AS count
FROM clean_cdr
GROUP BY calling_nbr, called_nbr
ORDER BY count

Running query in 'mysql+pyodbc://telecom'

12005982 rows affected.

,calling_nbr,called_nbr,count
0,njz85kbn,fm9461u4,1
1,njz85kbn,gcxrjn9k,1
2,njz85kbn,iagfexz5,1
3,njz85kbn,k82ejfxq,1
4,njz85kbn,m1nw9a1j,1
...,...,...,...
12005977,ec23fg9f,7wkrfg9c,1204
12005978,iq4xjsfp,itx04xbt,1226
12005979,a9gbfna5,5z4rfna3,1268
12005980,5z4rfna3,a9gbfna5,1746


In [19]:
%%sql

SELECT
    MAX(duration) AS column_value,
    decile
FROM (
    SELECT
        duration,
        NTILE(10) OVER (ORDER BY duration) AS decile
    FROM clean_cdr
) AS subquery
GROUP BY 
    decile

Running query in 'mysql+pyodbc://telecom'

10 rows affected.

,column_value,decile
0,13,1
1,19,2
2,25,3
3,32,4
4,41,5
5,53,6
6,71,7
7,104,8
8,186,9
9,7201,10


In [82]:
%config SqlMagic.named_parameters=True

In [35]:
%sql DROP TABLE clean_cdr

Running query in 'mysql+pyodbc://telecom'

""


In [36]:
%%sql

CREATE TABLE clean_cdr(
    SELECT
        SERV_ID                                                                     AS id,
        ACC_NBR                                                                     AS client_nbr, 
        CONVERT(SUBSTRING_INDEX(SUBSTRING_INDEX(END_TIME,' ',-1),':',1),UNSIGNED)   AS time,
        DAYOFWEEK(SUBSTRING_INDEX(END_TIME,' ',1))                                  AS day_of_week,
        CALLING_NBR                                                                 AS calling_nbr,
        DURATION                                                                    AS duration,
        CALLED_NBR                                                                  AS called_nbr, 
        ACCT_ITEM_TYPE_CODE                                                         AS acct_item_type_code,
        CELL_ID                                                                     AS cell_id
    FROM serv_acct_item_0838_201308
    WHERE 
        ACCT_ITEM_TYPE_CODE != -1                                                  AND
        (START_TIME LIKE "____/%/% %:__:__" AND END_TIME LIKE "____/%/% %:__:__")  AND
        (CALLING_NBR NOT LIKE 'NotMobile_%' AND CALLED_NBR NOT LIKE 'NotMobile_%') AND
        (CALLING_AREA_COdE != '-1' AND CALLED_AREA_CODE != '-1')                   AND
        (LAC != '-1' AND LAC != '0')                                               AND
        (CELL_ID != '-1' AND CELL_ID != '0')                                       AND
        (calling_nbr in (SELECT MSISDN FROM tb_asz_cdma_0838_201308))              AND
        (called_nbr in (SELECT MSISDN FROM tb_asz_cdma_0838_201308))
)

Running query in 'mysql+pyodbc://telecom'

13244425 rows affected.

""


- there is no effect of including or excluding the cell_id exception on area code

In [37]:
%%sql

CREATE TABLE clean_cdr_area(
    SELECT
        SERV_ID                                                                     AS id,
        ACC_NBR                                                                     AS client_nbr, 
        CONVERT(SUBSTRING_INDEX(SUBSTRING_INDEX(END_TIME,' ',-1),':',1),UNSIGNED)   AS time,
        DAYOFWEEK(SUBSTRING_INDEX(END_TIME,' ',1))                                  AS day_of_week,
        CALLING_NBR                                                                 AS calling_nbr,
        CALLING_AREA_CODE                                                           AS calling_area_code,
        DURATION                                                                    AS duration,
        CALLED_AREA_CODE                                                            AS called_area_code,
        CALLED_NBR                                                                  AS called_nbr, 
        ACCT_ITEM_TYPE_CODE                                                         AS acct_item_type_code,
        CELL_ID                                                                     AS cell_id
    FROM serv_acct_item_0838_201308
    WHERE 
        ACCT_ITEM_TYPE_CODE != -1                                                  AND
        (START_TIME LIKE "____/%/% %:__:__" AND END_TIME LIKE "____/%/% %:__:__")  AND
        (CALLING_NBR NOT LIKE 'NotMobile_%' AND CALLED_NBR NOT LIKE 'NotMobile_%') AND
        (CALLING_AREA_COdE != '-1' AND CALLED_AREA_CODE != '-1')                   AND
        (LAC != '-1' AND LAC != '0')                                               AND
        (calling_nbr in (SELECT MSISDN FROM tb_asz_cdma_0838_201308))              AND
        (called_nbr in (SELECT MSISDN FROM tb_asz_cdma_0838_201308))
)

Running query in 'mysql+pyodbc://telecom'

13244425 rows affected.

There's a new jupysql version available (0.10.0), you're running 0.9.0. To upgrade: pip install jupysql --upgrade


""


In [52]:
%sql DROP TABLE clean_cdr_

Running query in 'mysql+pyodbc://telecom'

""


In [38]:
%sql SELECT DISTINCT calling_area_code FROM clean_cdr_area

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,calling_area_code
0,0838


In [39]:
%sql SELECT DISTINCT called_area_code FROM clean_cdr_area

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,called_area_code
0,0838


- 0838: 四川德陽市
- 028: 成都
- 0816: 四川綿陽市
- 023: 重慶
- 010: 北京
- 021: 上海
- 0817: 四川南充市
- 0755: 廣東深圳市
- 0825: 四川遂寧市
- 0833: 四川樂山市
- 020: 廣州
- 029: 陝西西安、咸陽市
- 028: 成都、眉山、資陽(028M, 028Z)
- 0839: 四川廣元
- 0834: 四川涼山市
- 0832: 四川內江市(0832a)
- 0830: 四川瀘州市
- 0871: 雲南昆明市
- 0813: 四川自貢市

- 0831: 四川宜賓市
- 0835: 四川雅安市

In [44]:
%%sql

SELECT
    CALLING_AREA_CODE, COUNT(*)
FROM serv_acct_item_0838_201308
WHERE LAC = '-1'
GROUP BY CALLING_AREA_CODE
ORDER BY COUNT(*) DESC

Running query in 'mysql+pyodbc://telecom'

393 rows affected.

,CALLING_AREA_CODE,COUNT(*)
0,-1,65121659
1,0838,40712679
2,028,1257144
3,0816,251994
4,023,115762
...,...,...
388,0027,1
389,0041,1
390,0046,1
391,00855,1


In [45]:
%%sql

SELECT
    CALLING_AREA_CODE, COUNT(*)
FROM serv_acct_item_0838_201308
WHERE LAC = '0'
GROUP BY CALLING_AREA_CODE
ORDER BY COUNT(*) DESC

Running query in 'mysql+pyodbc://telecom'

395 rows affected.

,CALLING_AREA_CODE,COUNT(*)
0,0838,1666204
1,028,62928
2,023,49165
3,0991,20064
4,010,18985
...,...,...
390,00353,1
391,0041,1
392,00679,1
393,00855,1


In [49]:
%%sql

SELECT
    CALLING_AREA_CODE, COUNT(*)
FROM serv_acct_item_0838_201308
WHERE LAC = '0' AND CALLING_AREA_CODE LIKE '00%'
GROUP BY CALLING_AREA_CODE
ORDER BY COUNT(*) DESC

Running query in 'mysql+pyodbc://telecom'

63 rows affected.

,CALLING_AREA_CODE,COUNT(*)
0,00852,195
1,007,104
2,00886,65
3,001,52
4,0066,35
...,...,...
58,001418,1
59,00855,1
60,00352,1
61,00250,1


In [56]:
%%sql

SELECT
    COUNT(*)
FROM serv_acct_item_0838_201308
WHERE LAC = '0' AND CALLING_AREA_CODE = '0838' 

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,COUNT(*)
0,1666204


In [59]:
%%sql

SELECT
    COUNT(*)
FROM serv_acct_item_0838_201308
WHERE LAC = '0' AND CALLING_AREA_CODE = '0838' AND CALLED_AREA_CODE = '0838'

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,COUNT(*)
0,900108


In [60]:
%%sql

SELECT
    COUNT(*)
FROM serv_acct_item_0838_201308
WHERE 
    LAC != 0 AND LAC != -1 AND 
    CALLING_AREA_CODE != '0838' AND CALLED_AREA_CODE != '0838' AND 
    CALLING_AREA_CODE != -1 AND CALLED_AREA_CODE != '-1'

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,COUNT(*)
0,0


In [47]:
%%sql

SELECT
    CALLED_AREA_CODE, COUNT(*)
FROM serv_acct_item_0838_201308
WHERE LAC = '0'
GROUP BY CALLED_AREA_CODE
ORDER BY COUNT(*) DESC

Running query in 'mysql+pyodbc://telecom'

347 rows affected.

,CALLED_AREA_CODE,COUNT(*)
0,0838,1570579
1,028,74785
2,023,55833
3,010,21161
4,0991,20748
...,...,...
342,0033,1
343,0062,1
344,0065,1
345,00686,1


In [50]:
%%sql

SELECT
    CALLED_AREA_CODE, COUNT(*)
FROM serv_acct_item_0838_201308
WHERE LAC = '0' AND CALLED_AREA_CODE LIKE '00%'
GROUP BY CALLED_AREA_CODE
ORDER BY COUNT(*) DESC

Running query in 'mysql+pyodbc://telecom'

18 rows affected.

,CALLED_AREA_CODE,COUNT(*)
0,00852,24
1,001,15
2,0081,9
3,0061,8
4,00886,7
5,00853,6
6,0082,4
7,001514,4
8,0049,3
9,007,2


In [6]:
%sql DROP TABLE clean_cdr

Running query in 'mysql+pyodbc://telecom'

""


In [4]:
%config SqlMagic.named_parameters=True

In [7]:
%%sql

CREATE TABLE clean_cdr(
    SELECT
        CONVERT(SUBSTRING_INDEX(SUBSTRING_INDEX(END_TIME,' ',-1),':',1),UNSIGNED)   AS time,
        DAYOFWEEK(SUBSTRING_INDEX(END_TIME,' ',1))                                  AS day_of_week,
        CALLING_NBR                                                                 AS calling_nbr,
        CALLING_AREA_CODE                                                           AS calling_area_code, 
        DURATION                                                                    AS duration,
        CALLED_AREA_CODE                                                            AS called_area_code, 
        CALLED_NBR                                                                  AS called_nbr, 
        CELL_ID                                                                     AS cell_id
    FROM serv_acct_item_0838_201308
    WHERE 
        ACCT_ITEM_TYPE_CODE != -1                                                  AND
        (START_TIME LIKE "____/%/% %:__:__" AND END_TIME LIKE "____/%/% %:__:__")  AND
        (CALLING_NBR NOT LIKE 'NotMobile_%' AND CALLED_NBR NOT LIKE 'NotMobile_%') AND
        (CALLING_AREA_COdE != '-1' AND CALLED_AREA_CODE != '-1')                   AND
        (CELL_ID != '-1' AND CELL_ID != '0')                                       AND
        (CALLING_NBR in (SELECT MSISDN FROM tb_asz_cdma_0838_201308) OR CALLED_NBR in (SELECT MSISDN FROM tb_asz_cdma_0838_201308)) AND
        (CALLING_AREA_CODE = '0838' OR CALLED_AREA_CODE = '0838')
)

Running query in 'mysql+pyodbc://telecom'

52978357 rows affected.

""


In [8]:
%%sql 

SELECT calling_area_code, COUNT(*)
FROM clean_cdr
GROUP BY calling_area_code
ORDER BY COUNT(*) DESC

Running query in 'mysql+pyodbc://telecom'

358 rows affected.

,calling_area_code,COUNT(*)
0,0838,48903288
1,028,1859392
2,0816,397343
3,023,114454
4,0825,61418
...,...,...
353,0093,2
354,0036,1
355,0040,1
356,00880,1


In [9]:
%%sql

SELECT called_area_code, COUNT(*)
FROM clean_cdr
GROUP BY called_area_code
ORDER BY COUNT(*) DESC

Running query in 'mysql+pyodbc://telecom'

331 rows affected.

,called_area_code,COUNT(*)
0,0838,48485966
1,028,2266889
2,0816,465680
3,023,137094
4,0817,80407
...,...,...
326,0457,97
327,0464,62
328,0049,3
329,0062,2
